In [2]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

# Loading the data and training the neural network

In this part of the notebook nothing really interesting happens. It is just calling some build-in functions

Load the data 

In [4]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path

Build data block for fast.ai to work with

In [6]:
dls = ImageDataLoaders.from_folder(path, train = 'training', valid = 'testing') 

Train the neural network

In [8]:
learn = cnn_learner(dls, resnet18, pretrained=False, loss_func=F.cross_entropy, metrics=accuracy)
learn.fit_one_cycle(1, 0.1)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch,train_loss,valid_loss,accuracy,time
0,0.087064,0.667533,0.988000,11:33


[W ParallelNative.cpp:212] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:212] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:212] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:212] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:212] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

Now we export the model

In [9]:
learn.export(fname='export2.pkl', pickle_module=pickle, pickle_protocol=2)

# Build the pipeline

On the webpage we see a HTML-canvas where the user is able to write with his mouse. This HTML-canvas is transformed into a 'base64Field' which is basiclly a string representation of an image. This string gets decoded and with PIL.Image.open() it is transformed into an png image with python can read. 

This is the function which is my views.py file on the website. 

    def your_numbers(request):
        
        the_canvas       = request.POST.get('base64Field')
        base64_data      = re.sub('^data:image/.+;base64,', '', the_canvas)
        base64_data      = BytesIO(base64.b64decode(base64_data))
        written_numbers  = execute(base64_data)
        return render(request, 'your_numbers.html', {'written_numbers': written_numbers} )

The last function execute is definded below.


In [ ]:
from PIL import Image
import torch
import numpy
from fastai import *
from fastai.vision import *
from fastai.vision.all import *

execute() takes the png as input and return the numbers written. You can see that execute first transforms the image into tensor as fast.ai is based on that and it get the pixel position of the numbers in the png, as the user is able to write multiple numbers. This two functions are described below.

execute() then loades the 'learner' (the neural network which we trained before) and uses it to predict the class of the numbers in the png.

In [ ]:
def execute(img):

    img = Image.open(img).resize([100,28]).convert("P")
    to_predict = get_tensor(img, get_pixel_position_of_numbers(img))

    
    learn   = load_learner('/app/models_and_methods/methods/model/export2.pkl')
    preds,_ = learn.get_preds(dl=[to_predict])
    
    results = []

    for i in range(len(preds)):

        results.append(list(preds[i]).index(max(list(list(preds[i])))))

    return results

This function resizes the incomming png into the same as the png's we used to train our neural network. 

Note the argument "cut_here" tells get_tensor() where the nubers are located on the png.

In [ ]:
def get_tensor(img, cut_here):

    images = []
    quantity = int((len(cut_here)-2)/2)

    for i in range(quantity):
        number    = img.crop((cut_here[i*2], 0, cut_here[3+i*2], 28)).resize([28,28])
        images.append((torch.tensor([[250 if x == 1 else(x*5 if x > 1 else x) for x in this_list] for this_list in numpy.array(number)]).float()/255).repeat(1,3,1,1))

    return (torch.cat((images[0:quantity]), axis=0), torch.tensor([1] * quantity))

get_pixel_position_of_numbers() finds the position of the numbers on the png and it also find the amount of numbers whcih the user on the website has written. 

A png is basiccly a matrix with it's entries representing the inensity of a certain color of the png. In this case the intensity of black. therefore 0 = white and 256 = black. The algorithm checks every vertical line and checks whether the line is all white. if it is all white this means that there is nothing written. Next line. Next line. Next line... As soon as some line is not all 0 this means that there is something written. The column position gets saved. and from now of the algorithm look for the next time a line will be all white (0) again. This means that the number in the png ends there. 

In this fashion the alsorithm proceeds until the png ends.

In [ ]:
def get_pixel_position_of_numbers(img):
    
    pixels = list(numpy.array(img))
    position_of_pixel = []
    
    for i in range(100):
        
        is_pixel = False
        for j in range(28):
            if pixels[j][i] != 0:
                is_pixel = True
                break
        
        if not is_pixel and len(position_of_pixel)% 2 != 0:
            position_of_pixel.append(i)

        elif is_pixel and len(position_of_pixel)% 2 == 0:
            position_of_pixel.append(i)
    
    position_of_pixel.insert(0,0)
    position_of_pixel.append(100)
        
    return position_of_pixel 